# Qwen3-8B Model Fine-tuning
## TEKNOFEST 2025 - Eğitim Teknolojileri

Bu notebook, Qwen3-8B modelini 4-bit kuantizasyon ve LoRA ile fine-tune etmek için hazırlanmıştır.

In [ ]:
# Önce runtime'ı yeniden başlat ve temiz kurulum yap
!pip uninstall transformers accelerate -y
!pip install transformers==4.44.2 accelerate==0.34.2 bitsandbytes==0.43.3 peft==0.12.0 -q
!pip install sentencepiece protobuf -q

In [ ]:
# Cache temizliği
import os
import shutil

cache_dir = os.path.expanduser('~/.cache/huggingface')
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print("Cache temizlendi")

In [ ]:
# Runtime'ı yeniden başlatmak için
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Runtime yeniden başladıktan sonra buradan devam edin:

In [ ]:
# Gerekli kütüphaneleri import et
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("="*70)
print("MODEL YÜKLEME - Qwen3-8B")
print("="*70)

# Versiyon kontrolü
import transformers
print(f"Transformers version: {transformers.__version__}")

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"  # Daha stabil bir model
print(f"Model: {MODEL_ID}")

In [ ]:
# 4-bit kuantizasyon ayarları
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
print("✅ 4-bit kuantizasyon ayarlandı")

In [ ]:
# Tokenizer yükle
print("\n⏳ Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("✅ Tokenizer yüklendi")
print(f"   Vocab size: {len(tokenizer)}")

In [ ]:
# Model yükle
print("\n⏳ Model yükleniyor (5-10 dakika sürebilir)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    use_cache=False
)
print("✅ Model yüklendi (4-bit)")

In [ ]:
# LoRA konfigürasyonu
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Model'i LoRA için hazırla
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# İstatistikler
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())

print("\n📊 Model İstatistikleri:")
print(f"   Toplam parametre: {all_params:,}")
print(f"   Eğitilebilir: {trainable_params:,}")
print(f"   Eğitilebilir %: {100 * trainable_params / all_params:.2f}%")

In [ ]:
# GPU bellek durumu
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    print(f"\n💾 GPU Bellek:")
    print(f"   Kullanılan: {allocated:.2f} GB")
    print(f"   Rezerve: {reserved:.2f} GB")

print("\n✅ MODEL HAZIR")

## Alternatif: Daha Küçük Model Kullanımı

Eğer yukarıdaki model çok büyükse, daha küçük bir model kullanabilirsiniz:

In [ ]:
# Alternatif: Daha küçük Qwen modeli
# MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
# veya
# MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"